In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import animation as ibl_ani
import numbergen as ng
import imagen as ig
from tqdm import tqdm
import matplotlib.animation as animation
from IPython.display import HTML
import os
from os.path import join

In [7]:
def show(img, title=''):
    plt.figure(figsize=(15,10))
    plt.imshow(img)
    plt.title(title)
    plt.show()

In [45]:
test_folder = 'animation'
os.makedirs(test_folder, exist_ok=True)

animation_list = [join(test_folder, '{:03d}'.format(i)) for i in range(1, 30)]

In [46]:
def save_ibls(output_folder, num, size):
    animator = ibl_ani.base_ibl_animator(num, size)
    max_iter = 720
    
    ims = []
    fig = plt.figure()

    for i in range(max_iter):
        img = animator.animate_ibl(i, max_iter)
        fpath = join(output_folder, '{:06d}.png'.format(i))
        plt.axis('off')
        plt.imsave(fpath, img)
        im = plt.imshow(img, animated=True)
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=24, blit=True, repeat_delay=1000)
    ani.save('{}ibl_ani.mp4'.format(num))

In [47]:
for folder in tqdm(animation_list):
    os.makedirs(folder, exist_ok=True)
    folder_name = os.path.basename(folder)
    num = int(folder_name)
    save_ibls(folder, num, 0.01)





  0%|          | 0/29 [00:00<?, ?it/s]

pos: 
  [[256.  40.]]
vec: 
  [[0.49999232 0.86602984]]
size:
  [0.32468532]


<IPython.core.display.Javascript object>





  3%|▎         | 1/29 [00:35<16:32, 35.45s/it]

pos: 
  [[256.  40.]
 [256.  40.]]
vec: 
  [[ 0.89769072  0.44062611]
 [-0.53078906  0.84750397]]
size:
  [0.32468532 0.72925363]


...





 97%|█████████▋| 28/29 [42:26<02:18, 138.76s/it]

pos: 
  [[256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]
 [256.  40.]]
vec: 
  [[ 0.32801215  0.9446735 ]
 [-0.72873848 -0.68479211]
 [ 0.86354576  0.50427048]
 [ 0.80285178  0.59617869]
 [ 0.98136381 -0.19215896]
 [-0.82832553 -0.56024711]
 [-0.92642924  0.37646894]
 [-0.15704749  0.98759105]
 [-0.85096314  0.52522542]
 [-0.70195452 -0.71222177]
 [ 0.93181144 -0.36294274]
 [-0.9611316   0.27609066]
 [-0.99972806  0.02331947]
 [-0.27784217  0.96062674]
 [-0.98547364  0.16982844]
 [ 0.86017589  0.5099975 ]
 [ 0.67827261  0.73481036]
 [-0.05914061  0.99824966]
 [ 0.93733495 -0.3484296 ]
 [ 0.88665165  0.46243794]
 [-0.71083073  0.70336311]
 [-0.89046596  0.45504986]
 [-0.9106814

/home/ysheng/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


<IPython.core.display.Javascript object>





100%|██████████| 29/29 [44:43<00:00, 92.53s/it] 
